In [4]:
import pandas as pd
import sqlite3
import os
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Configuración visual
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# 2. Definir ruta y conectar
db_path = os.path.join('..', 'sql', 'heart_failure.db')

try:
    conn = sqlite3.connect(db_path)
    print(f"✅ Conexión establecida con éxito a: {db_path}")
except Exception as e:
    print(f"❌ Error de conexión: {e}")

# 3. Query de prueba
query_prueba = "SELECT * FROM pacientes LIMIT 5"
df_check = pd.read_sql_query(query_prueba, conn)

# 4. Mostrar
df_check.head()

✅ Conexión establecida con éxito a: ..\sql\heart_failure.db


,edad,anemia,cpk,diabetes,fraccion_eyeccion,presion_alta,plaquetas,creatinina_serica,sodio_serico,sexo,fumador,dias_seguimiento,muerte
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
